# 5-OpenJij GPU

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OpenJij/OpenJijTutorial/blob/master/source/ja/5_OpenJijGPU.ipynb)

このセクションではGPUを用いてSimulated Quantum Annealing(SQA)を行う方法をご紹介します。

## Google Colabでの設定とインストール

一般のPCにはCUDA環境が整っていない、もしくはCUDAに対応するNVIDIAのGPUが搭載されていないこともあります。しかし**Google colaboratory**を使えば誰でもGPGPUを使うことができます。

### GPU設定

GPU実行したいJupyter NotebookをGoogle Colabで開きます。  

```
ランタイム > ランタイムのタイプを変更 > ハードウェアアクセラレータをGPUに選択
```

この設定を行うことでGPUを用いたJupyter Notebookの実行が可能となります。

### OpenJijのインストール

Google ColabにプリインストールされているCMakeは古いバージョンです。まずはそれをバージョンアップしましょう。

In [ ]:
!pip install -U cmake

その後、OpenJijをインストールします。

In [ ]:
!pip install openjij

## GPUSQA計算

OpenJijのGPUでサポートしているのはキメラグラフ上でのモンテカルロ計算です。  
まずは GPUSQASampler のインスタンスを作ります。

In [3]:
import openjij as oj

sampler = oj.GPUSQASampler(num_reads=100, unit_num_L=2)

ここで `unit_num_L` はキメラユニットセルが並ぶ2次元格子の1辺の長さを指定します。  
上の例では 2×2=4つのキメラユニットセルが並んでいる系を作成しています。キメラユニットセルは8bitで1つを構成しているため、総ビット数は
  
${\rm total\_num} = {\rm unit\_num\_L}^2 \times 8$

のように計算されます。

GPUSQASamplerでは、のちに sample_qubo などで少ない変数を指定しても、ここで作ったキメラグラフの系のサイズだけSQA計算を行います。  
`unit_num_L`は無駄に大きすぎないように注意しましょう。また`unit_num_L`はコンストラクタで指定しなくても、`sample_qubo`を実行するときの引数としても指定できます。

## 実行

ではSQAを実行してみましょう。これまで紹介してきたSamplerと同様に、sample_qubo (or sample_ising) で計算を行うことができます。
ただし、キメラグラフを満たさない添字の相互作用を指定するとエラーになります。

In [4]:
Q = {(0, 4): -1, (0, 5): -1, (4, 12): 1}
response = sampler.sample_qubo(Q)
response

Response(rec.array([([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0

## 全結合模型のキメラグラフへの埋め込み

これまで説明したものは、相互作用がキメラグラフ上に存在するもののみの計算でした。以下では全結合の問題のように、相互作用が複雑なものをキメラグラフに埋め込む場合のスクリプトをご紹介します。  
キメラグラフへの埋め込みを行うために、`dwave`の`EmbeddingComposite`を併用しましょう。`pip`を用いて`dwave-system`をインストールします。

In [ ]:
!pip install dwave-system

そしてsampler部分のスクリプトを以下のように書き換えます。

In [15]:
from dwave.system.composites import EmbeddingComposite

gpusqa = oj.GPUSQASampler(num_reads=100, unit_num_L=2)
sampler = EmbeddingComposite(gpusqa)